In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import os
import pickle


In [2]:

dataset_dir = r'C:\Users\kara\Desktop\Project\Flower\dataset\organized'
train_dir = os.path.join(dataset_dir, 'train')
valid_dir = os.path.join(dataset_dir, 'valid')
model_save_path = "flower_model.pth"
labels_save_path = "flower_labels.pkl"
batch_size = 32
num_epochs = 10
learning_rate = 0.001

In [3]:

# Check GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [4]:

# Data transforms
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_valid = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [5]:

# Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=transform_train)
valid_dataset = datasets.ImageFolder(valid_dir, transform=transform_valid)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)


In [6]:

# Save class labels (flower names)
class_to_idx = train_dataset.class_to_idx
idx_to_class = {v: k for k, v in class_to_idx.items()}
with open(labels_save_path, "wb") as f:
    pickle.dump(idx_to_class, f)

print(f"Number of classes: {len(idx_to_class)}")
print(f"Classes: {idx_to_class}")


Number of classes: 102
Classes: {0: 'alpine sea holly', 1: 'anthurium', 2: 'artichoke', 3: 'azalea', 4: 'ball moss', 5: 'balloon flower', 6: 'barbeton daisy', 7: 'bearded iris', 8: 'bee balm', 9: 'bird of paradise', 10: 'bishop of llandaff', 11: 'black-eyed susan', 12: 'blackberry lily', 13: 'blanket flower', 14: 'bolero deep blue', 15: 'bougainvillea', 16: 'bromelia', 17: 'buttercup', 18: 'californian poppy', 19: 'camellia', 20: 'canna lily', 21: 'canterbury bells', 22: 'cape flower', 23: 'carnation', 24: 'cautleya spicata', 25: 'clematis', 26: "colt's foot", 27: 'columbine', 28: 'common dandelion', 29: 'corn poppy', 30: 'cyclamen', 31: 'daffodil', 32: 'desert-rose', 33: 'english marigold', 34: 'fire lily', 35: 'foxglove', 36: 'frangipani', 37: 'fritillary', 38: 'garden phlox', 39: 'gaura', 40: 'gazania', 41: 'geranium', 42: 'giant white arum lily', 43: 'globe thistle', 44: 'globe-flower', 45: 'grape hyacinth', 46: 'great masterwort', 47: 'hard-leaved pocket orchid', 48: 'hibiscus', 4

In [7]:

# Load pre-trained ResNet-18 model
model = models.resnet18(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(idx_to_class))
model = model.to(device)


c:\Users\kara\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\kara\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [9]:

# Training loop
def train_model(model, criterion, optimizer, num_epochs):
    best_val_loss = float("inf")
    for epoch in range(num_epochs):

        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

        train_loss = running_loss / len(train_dataset)
        train_acc = correct / total

        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for images, labels in valid_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * images.size(0)
                _, predicted = outputs.max(1)
                val_correct += predicted.eq(labels).sum().item()
                val_total += labels.size(0)

        val_loss = val_loss / len(valid_dataset)
        val_acc = val_correct / val_total

        print(f"Epoch [{epoch+1}/{num_epochs}] - "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), model_save_path)
            print(f"Best model saved with Val Loss: {val_loss:.4f}")

    return model

print("Starting training...")
model = train_model(model, criterion, optimizer, num_epochs)

Starting training...
Epoch [1/10] - Train Loss: 2.3185, Train Acc: 0.4518, Val Loss: 1.9729, Val Acc: 0.4647
Best model saved with Val Loss: 1.9729
Epoch [2/10] - Train Loss: 1.2849, Train Acc: 0.6580, Val Loss: 1.2489, Val Acc: 0.6490
Best model saved with Val Loss: 1.2489
Epoch [3/10] - Train Loss: 0.8948, Train Acc: 0.7570, Val Loss: 1.0405, Val Acc: 0.7167
Best model saved with Val Loss: 1.0405
Epoch [4/10] - Train Loss: 0.8050, Train Acc: 0.7788, Val Loss: 0.7619, Val Acc: 0.7912
Best model saved with Val Loss: 0.7619
Epoch [5/10] - Train Loss: 0.7148, Train Acc: 0.8029, Val Loss: 0.5736, Val Acc: 0.8402
Best model saved with Val Loss: 0.5736
Epoch [6/10] - Train Loss: 0.6423, Train Acc: 0.8227, Val Loss: 0.4284, Val Acc: 0.8824
Best model saved with Val Loss: 0.4284
Epoch [7/10] - Train Loss: 0.5517, Train Acc: 0.8507, Val Loss: 0.6048, Val Acc: 0.8284
Epoch [8/10] - Train Loss: 0.5319, Train Acc: 0.8535, Val Loss: 0.4902, Val Acc: 0.8794
Epoch [9/10] - Train Loss: 0.5070, Train 